In [ ]:
# Установка библиотек
!pip install diffusers transformers accelerate safetensors torchvision matplotlib
!pip install peft  # Для LoRA (если нужно)

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

# Загрузка модели (может занять 5-10 минут)
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

# Генерация изображения
prompt = "A realistic cat with green eyes, 4K, detailed fur"
image = pipe(prompt).images[0]
image.save("sdxl_output.png")
image  # Показать изображение

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import save_image
import os
from pycocotools.coco import COCO
from PIL import Image
import requests
from io import BytesIO
import random

# Гиперпараметры
batch_size = 128
lr = 0.0002
latent_dim = 100
img_size = 64
channels = 3
epochs = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Создание папок
os.makedirs("gan_images", exist_ok=True)
os.makedirs("data/coco", exist_ok=True)

# Трансформации
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Класс для загрузки COCO
class CocoDataset(torch.utils.data.Dataset):
    def __init__(self, root, annFile, transform=None):
        self.root = root
        self.coco = COCO(annFile)
        self.ids = list(self.coco.imgs.keys())
        self.transform = transform
        
    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        img_info = coco.loadImgs(img_id)[0]
        img_url = img_info['coco_url']
        
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content)).convert('RGB')
        
        if self.transform is not None:
            img = self.transform(img)
            
        return img, 0  # Возвращаем 0 как фиктивный лейбл
    
    def __len__(self):
        return len(self.ids)

# Загрузка COCO (если файлов нет, они скачаются автоматически)
if not os.path.exists("data/coco/annotations_trainval2017.zip"):
    !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P data/coco
    !unzip data/coco/annotations_trainval2017.zip -d data/coco

# Создаем датасет (будет загружать изображения по URL)
dataset = CocoDataset(
    root="data/coco",
    annFile="data/coco/annotations/instances_train2017.json",
    transform=transform
)

# Фильтруем датасет (берем только первые 5000 изображений для ускорения)
dataset.ids = dataset.ids[:5000]
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Генератор
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

# Дискриминатор
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x).view(-1)

# Инициализация моделей
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Оптимизаторы
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Функция потерь
criterion = nn.BCELoss()

# Фиксированный шум для визуализации
fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

# Обучение
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Реальные и фейковые метки
        real = torch.ones(imgs.size(0), device=device)
        fake = torch.zeros(imgs.size(0), device=device)
        
        real_imgs = imgs.to(device)
        
        # Обучение дискриминатора
        optimizer_D.zero_grad()
        
        # Потери на реальных изображениях
        output_real = discriminator(real_imgs)
        loss_real = criterion(output_real, real)
        
        # Генерация фейковых изображений
        noise = torch.randn(imgs.size(0), latent_dim, 1, 1, device=device)
        fake_imgs = generator(noise)
        
        # Потери на фейковых изображениях
        output_fake = discriminator(fake_imgs.detach())
        loss_fake = criterion(output_fake, fake)
        
        # Общие потери дискриминатора
        loss_D = (loss_real + loss_fake) / 2
        loss_D.backward()
        optimizer_D.step()
        
        # Обучение генератора
        optimizer_G.zero_grad()
        
        # Потери генератора
        output = discriminator(fake_imgs)
        loss_G = criterion(output, real)
        loss_G.backward()
        optimizer_G.step()
        
        # Логирование
        if i % 10 == 0:
            print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(dataloader)}] [D loss: {loss_D.item():.4f}] [G loss: {loss_G.item():.4f}]")
    
    # Сохранение сгенерированных изображений
    if epoch % 5 == 0:
        with torch.no_grad():
            fake = generator(fixed_noise).detach().cpu()
        save_image(fake, f"gan_images/epoch_{epoch}.png", nrow=8, normalize=True)

# Сохранение весов генератора
torch.save(generator.state_dict(), "generator_coco.pth")

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Генератор
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

# Загрузка предобученного генератора (пример весов)
generator = Generator().to("cuda")
generator.load_state_dict(torch.load("generator_coco.pth"))  

In [ ]:
# Преобразование изображения от SDXL для DCGAN
def preprocess_image(pil_image, size=64):
    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    return transform(pil_image).unsqueeze(0).to("cuda")

# Улучшение изображения
input_image = preprocess_image(image)
with torch.no_grad():
    enhanced = generator(input_image)

# Визуализация
def show_image(tensor):
    tensor = tensor.squeeze().permute(1, 2, 0).cpu().numpy()
    tensor = (tensor * 127.5 + 127.5).astype("uint8")
    plt.imshow(tensor)
    plt.axis("off")
    plt.show()

print("Original (SDXL):")
show_image(input_image)

print("Enhanced (DCGAN):")
show_image(enhanced)

In [ ]:
# Дополнительные импорты
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Загрузка COCO (пример)
!wget http://images.cocodataset.org/zips/train2017.zip
!unzip train2017.zip

# Датасет и загрузчик
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
dataset = ImageFolder("train2017", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Обучение
def train_dcgan(epochs=10):
    generator = Generator().to("cuda")
    discriminator = Discriminator().to("cuda")
    opt_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    opt_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        for real_imgs, _ in dataloader:
            real_imgs = real_imgs.to("cuda")
            noise = torch.randn(real_imgs.size(0), 100, 1, 1, device="cuda")
            fake_imgs = generator(noise)

            # Обновление дискриминатора
            opt_d.zero_grad()
            real_loss = criterion(discriminator(real_imgs), torch.ones_like(discriminator(real_imgs)))
            fake_loss = criterion(discriminator(fake_imgs.detach()), torch.zeros_like(discriminator(fake_imgs)))
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            opt_d.step()

            # Обновление генератора
            opt_g.zero_grad()
            g_loss = criterion(discriminator(fake_imgs), torch.ones_like(discriminator(fake_imgs)))
            g_loss.backward()
            opt_g.step()

        print(f"Epoch {epoch}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

    torch.save(generator.state_dict(), "generator.pth")

train_dcgan(epochs=5)  

In [ ]:
# Сохранить генератор
torch.save(generator.state_dict(), "generator.pth")

# Загрузить в будущем
generator = Generator().to("cuda")
generator.load_state_dict(torch.load("generator.pth"))